# Prequisites

In [1]:
%pip install move-ugc-python

Looking in indexes: https://pypi.python.org/simple

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Import all requires modules

In [42]:
from datetime import datetime
import os
from pathlib import Path
import time
import requests
from move_ugc import MoveUgc
from move_ugc.schemas.sources import SourceIn
from move_ugc.schemas.sync_method import SyncMethodInput
from move_ugc.schemas.volume import AreaType

# Declare utility functions

In [43]:
class MoveAI:
    """MoveAI UGC utility class."""

    def __init__(self, api_key, endpoint_url=None) -> None:
        """Initialize MoveAI UGC utility class.

        Args:
            api_key: API key.
            endpoint_url: Endpoint
        """
        self.api_key = api_key
        if endpoint_url is None:
            endpoint_url = 'https://api.move.ai/ugc/graphql'
        self.endpoint_url = endpoint_url
        self.client = MoveUgc(api_key=api_key, endpoint_url=endpoint_url)

    def create_files(self, video_path: str) -> str:
        """Create a file in MoveUGC.

        Args:
            video_path: Path to the video file.

        Returns:
            str: File ID.
        """
        video_file = self.client.files.create(file_type="mp4")

        with open(video_path, 'rb') as f:
            requests.put(video_file.presigned_url, data=f.read())

        return video_file.id

    def create_volume(
        self, sources, human_height: float, name=None, metadata=None,
    ):
        """Create a new volume.

        Args:
            sources: List of sources.
            human_height: Human height.
            name: Name of the volume.
            metadata: Metadata.

        Returns:
            VolumeType
        """
        if metadata is None:
            metadata = {"test": "test"}
        return self.client.volumes.create_human_volume(
            sources=sources,
            name=name,
            metadata=metadata,
            human_height=human_height,
            area_type=AreaType.NORMAL,
        )

    def get_volume(self, volume_id: str):
        """Retrieve volume.

        Args:
            VolumeType
        """
        return self.client.volumes.retrieve_human_volume(id=volume_id)

    def create_take(self, sources, volume_id: str, sync_method, name=None, metadata=None) -> str:
        """Create a new take.

        Args:
            sources: List of sources.
            volume_id: Volume ID.
            name: Name of the take.
            metadata: Metadata.

        Returns:
            str: Take ID.
        """
        if metadata is None:
            metadata = {"test": "test"}
        return self.client.takes.create_multicam(
            volume_id=volume_id,
            sources=sources,
            metadata=metadata,
            name=name,
            sync_method=sync_method,
        ).id

    def create_job(
        self, take_id: str, number_of_actors: str, name=None, metadata=None,
    ):
        """Create a new multicam job.

        Args:
            take_id: Take ID.
            number_of_actors: Number of actors.
            name: Name of the job.
            metadata: Metadata.

        Returns:
            JobType.
        """
        return self.client.jobs.create_multicam(
            take_id=take_id, number_of_actors=number_of_actors, metadata={"test": "test_job"},
        )

    def get_job(self, job_id, expand=False):
        # Get a job using the Move One Public API
        # Implement job retrieval logic using move_ugc_python SDK
        if expand is False:
            job = self.client.jobs.retrieve(id=job_id)
        else:
            job = self.client.jobs.retrieve(
                id=job_id, expand=["take", "outputs", "client"]
            )
        return job
    
    def download_outputs(self, job_id, output_dir, output_name):
        # make output dir if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        # get job
        job = self.get_job(job_id, expand=True)
        # for each output download the file in the output_dir
        output_paths = []
        for output in job.outputs:
            output_file_name = f"{output_name}{output.file.type}"
            output_path = os.path.join(output_dir, output_file_name)
            with open(output_path, 'wb') as f:
                response = requests.get(output.file.presigned_url)
                f.write(response.content)
            output_paths.append(output_path)
        return output_paths

# Instantiate utility class

In [44]:
client = MoveAI(os.environ.get('MOVE_API_KEY', '<Your Move AI API key if it is not set as environment variable>'))
if not client.api_key or client.api_key == '<Your Move AI API key if it is not set as environment variable>':
    raise ValueError('Please set the MOVE_API_KEY environment variable or pass it as an argument to MoveAI')

# Create volume

## 1. Create required video files for volumes 

In [46]:
sources = []
# Create two files
for camera_number in range(1,3):
    # Creates and uploads cam01_volume.mp4, cam0N_volume.mp4 etc.
    input_video_file = Path(f'data/input_videos/cam0{camera_number}_calib.mp4')
    ugc_file = client.create_files(input_video_file)
    sources.append(
        SourceIn(
            device_label=f"cam0{camera_number}",
            file_id=ugc_file,
            format="MP4",
            camera_settings={
                "lens": "goprohero10-fhd",
            }
        )
    )

## 2. Create volume and wait for it to be processed

In [47]:
volume = client.create_volume(sources=sources, human_height=1.77, name="Test volume")
# Poll the volume until it is finished processing
attempts = 0

while attempts < 100:
    volume = client.get_volume(volume.id)
    update_str = f"[{datetime.now().isoformat()} | {attempts}] Volume {volume.id} is {volume.state}"
    print(update_str)
    if volume.state == 'FINISHED':
        print("Volume is processed successfully, please proceed to job creation")
        break
    else:
        time.sleep(30)
        attempts += 1

[2024-10-07T17:03:16.378060 | 0] Volume volume-43968e7e-5141-468d-81e5-c13a43dac5a8 is RUNNING
[2024-10-07T17:03:47.717917 | 1] Volume volume-43968e7e-5141-468d-81e5-c13a43dac5a8 is RUNNING
[2024-10-07T17:04:18.532382 | 2] Volume volume-43968e7e-5141-468d-81e5-c13a43dac5a8 is RUNNING
[2024-10-07T17:04:49.385753 | 3] Volume volume-43968e7e-5141-468d-81e5-c13a43dac5a8 is RUNNING
[2024-10-07T17:05:20.283078 | 4] Volume volume-43968e7e-5141-468d-81e5-c13a43dac5a8 is RUNNING
[2024-10-07T17:05:51.382649 | 5] Volume volume-43968e7e-5141-468d-81e5-c13a43dac5a8 is FINISHED
Volume is processed successfully, please proceed to job creation


# Create a multicam take

## 1. Create required video files for multicam take

In [48]:
sources = []
for camera_number in range(1,3):
    # Creates and uploads cam01_volume.mp4, cam0N_volume.mp4 etc.
    input_video_file = Path(f'data/input_videos/cam0{camera_number}_action.mp4')
    ugc_file = client.create_files(input_video_file)
    sources.append(
        SourceIn(
            device_label=f"cam0{camera_number}",
            file_id=ugc_file,
            format="MP4",
            camera_settings={
                "lens": "goprohero10-fhd",
            }
        )
    )

## 2. Create a multicam take

In [49]:
take_id = client.create_take(
    sources, 
    volume_id=volume.id, 
    sync_method=SyncMethodInput(
        clap_window={
            "start_time": 2,
            "end_time": 4,
        },
    ),
    name="Test take"
)

## 3. Create a multicam job

In [50]:
job = client.create_job(take_id=take_id, number_of_actors=1, name="Test job")

## 4. Wait until the job is finished and get the outputs

In [51]:
# Poll the job until it is finished
attempts = 0
output_dir = Path('data/output')
while attempts < 100:
    job = client.get_job(job.id)
    update_str = f"[{datetime.now().isoformat()} | {attempts}] Job {job.id} is {job.state}"
    print(update_str)
    if job.state == 'FINISHED':
        outputs = client.download_outputs(job.id, output_dir, input_video_file.stem)
        print(f"Outputs downloaded to {output_dir}")
        print(f"Output files: {outputs}")
        break
    else:
        time.sleep(30)
        attempts += 1

[2024-10-07T17:06:22.237643 | 0] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:06:52.953168 | 1] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:07:23.779433 | 2] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:07:54.395680 | 3] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:08:25.422643 | 4] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:08:55.974503 | 5] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:09:26.656823 | 6] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:09:57.278859 | 7] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:10:27.891841 | 8] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:10:58.536273 | 9] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:11:29.135036 | 10] Job job-bc44cdba-6ac8-4dc0-a9ec-4494986c7144 is RUNNING
[2024-10-07T17:11:59